In [23]:
import os
import gc
import sys
import pickle
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score, roc_auc_score
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from transformers import TFAutoModelForImageClassification
from keras.models import load_model
import pandas as pd
import numpy as np

In [21]:
def setup_gpu():
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
            tf.config.experimental.set_memory_growth(gpus[0], True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(f"Physical GPUs: {gpus}")
            print(f"Logical GPUs: {logical_gpus}")
        except RuntimeError as e:
            print(e)
def load_data_from_pickle(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f)
    images, labels, infos = data
    return images, labels, infos

def generate_dataset(start, end, train_images, train_labels, batch_size=16):
    images = train_images[start:end]
    labels = train_labels[start:end]
    
    if images.shape[-1] == 1:
        images = np.repeat(images, 3, axis=-1)
    
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    return dataset.batch(batch_size)

def create_prediction_label_df(test_pred, test_labels, test_infos):
    data = {
        'subject_id': [info['subject_id'] for info in test_infos],
        'age': [info['age'] for info in test_infos],
        'race': [info['race'] for info in test_infos],
        'gender': [info['gender'] for info in test_infos],
    }

    # Assuming each element in test_pred and test_labels is a list/array
    for i in range(len(test_pred[0])):
        data[f'pred_{i}'] = [pred[i] for pred in test_pred]
        data[f'label_{i}'] = [label[i] for label in test_labels]

    return pd.DataFrame(data)

In [3]:

model = load_model('./models/resnetv0/resnet_model_0.keras')

2024-08-12 22:52:58.793622: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21339 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6
2024-08-12 22:52:58.794102: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 1604 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:b3:00.0, compute capability: 8.6


In [13]:
test_images, test_labels, test_infos= load_data_from_pickle('./new_data/test_data.pkl')

In [5]:
test_images = np.array(test_images)
test_labels = np.array(test_labels)
if test_images.shape[-1] != 3:
    test_images = np.repeat(test_images, 3, axis=-1)

In [6]:
print(len(test_images))

58386


In [7]:
test_pred1 = model.predict(test_images[:12000], batch_size = 16)

2024-08-12 22:53:36.431532: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907


750/750 [==============================] - 15s 17ms/step


In [8]:
test_pred2 = model.predict(test_images[12000:24000], batch_size = 16)

750/750 [==============================] - 13s 17ms/step


In [10]:
test_pred3 = model.predict(test_images[24000:], batch_size = 4)

8597/8597 [==============================] - 100s 12ms/step


In [11]:
test_pred = np.concatenate((test_pred1, test_pred2, test_pred3), axis=0)

In [17]:
test_pred[0]

array([0.0561431 , 0.01501172, 0.00117542, 0.00426925, 0.09128474,
       0.7594785 ], dtype=float32)

In [18]:
test_labels[0]

[0, 0, 0, 0, 0, 1]

In [19]:
test_infos[0]

{'gender': 'F', 'age': 68, 'race': 'WHITE', 'subject_id': 11812752}

In [24]:
df = create_prediction_label_df(test_pred, test_labels, test_infos)

In [25]:
df

,subject_id,age,race,gender,pred_0,label_0,pred_1,label_1,pred_2,label_2,pred_3,label_3,pred_4,label_4,pred_5,label_5
0,11812752,68,WHITE,F,0.056143,0,0.015012,0,0.001175,0,0.004269,0,0.091285,0,0.759479,1
1,11812752,68,WHITE,F,0.051060,0,0.006028,0,0.001799,0,0.006515,0,0.050387,0,0.863849,1
2,11812752,68,WHITE,F,0.021372,0,0.004059,0,0.000743,0,0.004475,0,0.092967,0,0.849508,1
3,11812752,68,WHITE,F,0.031842,0,0.001862,0,0.000797,0,0.006196,0,0.047191,0,0.934808,1
4,15197921,-1,,,0.034598,0,0.010697,0,0.000503,0,0.005243,0,0.077575,0,0.871780,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58381,11183154,69,WHITE,M,0.104890,0,0.011576,0,0.001189,0,0.003390,0,0.057218,0,0.851656,1
58382,16736626,46,WHITE,M,0.099216,0,0.071405,0,0.004492,0,0.001722,0,0.200777,0,0.613287,1
58383,16736626,46,WHITE,M,0.439834,0,0.023786,0,0.000858,0,0.007120,0,0.112077,0,0.549323,1
58384,16736626,46,WHITE,M,0.446967,0,0.284390,0,0.009412,0,0.022791,0,0.294759,0,0.150314,0


In [28]:
df.to_csv('./new_data/Test_set_prediction.csv', index=False)